In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from cleaner_utilities import MyDataTransformer
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv( '~/NLP/NLP_Disaster_tweets/train.csv')
X_train, X_val, y_train, y_val= train_test_split(train_df, train_df['target'])

my_data_transformer = MyDataTransformer()

y_train = np.array(y_train).reshape(1,-1)

X_train = my_data_transformer.fit_transform(X_train)

X_train.shape

In [7]:
import tensorflow.compat.v1 as tf

In [8]:
tf.disable_eager_execution()

In [9]:
X = tf.placeholder(dtype = tf.float32 , shape=[X_train.shape[0], None], name='X')
Y = tf.placeholder(dtype = tf.float32,  shape=[1,None], name='Y')

W1 = tf.get_variable('W1', shape=[256,X_train.shape[0]], initializer=tf.initializers.glorot_uniform())
b1 = tf.get_variable('b1', shape=[256,1],initializer=tf.zeros_initializer() )

W2 = tf.get_variable('W2', shape=[128,256], initializer=tf.initializers.glorot_uniform())
b2 = tf.get_variable('b2', shape=[128,1],initializer=tf.zeros_initializer() )

W3 = tf.get_variable('W3', shape=[1,128], initializer=tf.initializers.glorot_uniform())
b3 = tf.get_variable('b3', shape=[1,1],initializer=tf.zeros_initializer() )



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
Z1 = tf.add(tf.matmul(W1,X),b1)
A1 = tf.nn.relu(Z1)
Z2 = tf.add(tf.matmul(W2,A1),b2)
A2 = tf.nn.relu(Z2)
Z3 = tf.add(tf.matmul(W3,A2),b3)

In [11]:

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Z3,labels=Y))

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [13]:
init = tf.global_variables_initializer()

In [14]:
session = tf.Session()
session.run(init)

In [15]:
for i in range(30):
    
    _,train_cost = session.run([optimizer,cost], feed_dict={X:X_train,Y:y_train})
    
    print(f'training cost at iteration {i} = {train_cost}')



training cost at iteration 0 = 0.866280198097229
training cost at iteration 1 = 4.733925819396973
training cost at iteration 2 = 0.2944730818271637
training cost at iteration 3 = 0.13008588552474976
training cost at iteration 4 = 0.1748783141374588
training cost at iteration 5 = 0.18408310413360596
training cost at iteration 6 = 0.1195172592997551
training cost at iteration 7 = 0.09646999090909958
training cost at iteration 8 = 0.08143307268619537
training cost at iteration 9 = 0.07061268389225006
training cost at iteration 10 = 0.06253376603126526
training cost at iteration 11 = 0.05638793855905533
training cost at iteration 12 = 0.0517449714243412
training cost at iteration 13 = 0.04810613393783569
training cost at iteration 14 = 0.0450831763446331
training cost at iteration 15 = 0.0425918772816658
training cost at iteration 16 = 0.040540795773267746
training cost at iteration 17 = 0.038775693625211716
training cost at iteration 18 = 0.037240251898765564
training cost at iteration 19

In [16]:
X_val = my_data_transformer.transform(X_val)
y_val = np.array(y_val).reshape(1,-1)

In [19]:
def print_validation_accuracy():
    
    z3 = session.run(Z3,feed_dict={X:X_val,Y:y_val})

    y_val_hat = tf.nn.sigmoid(z3)

    y_val_hat = session.run(y_val_hat)

    y_val_hat = np.array(y_val_hat)

    val_preds = np.where(y_val_hat >= 0.5,1,0)

    print(accuracy_score(y_val[0],val_preds[0]))

In [21]:
print_validation_accuracy()

0.7531512605042017


In [22]:
#currently it's overfitting need to work on MyDataTransformer which is data cleaning part